In [35]:
import json
import pandas as pd
import numpy as np
import sys
import random
from collections import defaultdict
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
sys.path.append('./DnD-Battler/')
import DnD

In [2]:
url = "https://raw.githubusercontent.com/oganm/dnddata/master/data-raw/dnd_chars_all.json"
df = pd.read_json(url)
df = df.transpose()

In [3]:
ignored = ['ip', 'finger', 'date', 'hash', 'choices', 'name', 'country']
dont_process = ['weapons', 'spells', 'skills']
clean_df = pd.json_normalize(df['name'])
clean_df = clean_df['alias'].str[0]

for key in df.keys():
    if key not in ignored:
        if key in dont_process:
            data = df[key].reset_index(drop=True)
        else:
            try: # try doing a json_normalize, which will fail if it's not a json
                data = pd.json_normalize(df[key])
                for col in data.keys(): # unpack the singletons in each json column
                    data[col] = data[col].str[0]
            except AttributeError: 
                data = df[key].str[0]
                data = data.reset_index(drop=True) # use integer index to conform with json_normalize index

        clean_df = pd.concat([clean_df, data], axis=1)

num_spells = df['spells'].apply(len).reset_index(drop=True)
clean_df['num_spells'] = num_spells

# add generic "primary class" column based on highest level class of character
class_list = ['Artificer.level','Barbarian.level','Bard.level','Cleric.level','Druid.level','Fighter.level','Monk.level','Paladin.level','Ranger.level','Rogue.level','Sorcerer.level','Warlock.level','Wizard.level']
class_df = clean_df[class_list]

primary_class = class_df.idxmax(axis=1)
class1 = list(primary_class.str.split('.').str[0])
clean_df['class1'] = class1

# add generic "primary class level" column/label
primary_class_level = class_df.max(axis=1)
clean_df['level1'] = primary_class_level

# one-hot encode castingStat
castingStat = pd.get_dummies(clean_df["castingStat"], prefix='castingStat')
clean_df = pd.concat([clean_df, castingStat], axis = 1)

# one-hot encode processedRace 
processedRace = pd.get_dummies(clean_df["processedRace"], prefix='processedRace')
clean_df = pd.concat([clean_df, processedRace], axis = 1)

# add hp / level
clean_df['hp_per_lvl'] = clean_df['HP'] / clean_df['level1']

In [15]:
level_groups = ['1-3', '4-7', '8-11', '12-15', '16-18', '19-20']
weapon_info = {'club': 4, 'greatclub': 8,
                           'dagger': 4, 'shortsword': 6, 'longsword': 8, 'bastardsword': 10, 'greatsword': 12,
                           'rapier': 8, 'scimitar': 6, 'sickle': 4,
                           'handaxe': 6, 'battleaxe': 8, 'waraxe': 10, 'greataxe': 12,
                           'javelin': 6, 'spear': 6, 'flail': 8, 'glaive': 10, 'halberd': 10, 'lance': 12, 'pike': 10,
                           'trident': 6,
                           'war pick': 8,
                           'lighthammer': 4, 'mace': 6, 'warhammer': 8,
                           'quaterstaff': 6, 'morningstar': 8, 'punch': 1,
                           'whip': 4}

char_list = {
    '1-3': [],
    '4-7': [], 
    '8-11': [], 
    '12-15': [], 
    '16-18': [], 
    '19-20': []
}

iterations = 500

# clean_df[~clean_df['levelGroup'].isin(levelGroups)]['levelGroup']
for group in level_groups:
    grouped_chars = clean_df[clean_df['levelGroup'] == group]
    
    for n in range(iterations):
#         group_size = random.randint(2, 6)
        group_size = 1

        samples = [grouped_chars.sample(group_size), grouped_chars.sample(group_size)]
        chars = []

        for g in range(len(samples)):
            sample = samples[g]
            group_chars = []

            for i in range(len(sample)):
                char = sample.iloc[i]
                weapons = char['weapons']
                weapon_list = []
                for weapon in weapons:
                    if weapons[weapon]['processedWeapon'][0] is not None:
                        weapon_list.append(weapons[weapon]['processedWeapon'][0].lower())

                best_weapon = 'punch'
                for weapon in weapon_list:
                    if weapon in weapon_info and weapon_info[weapon] > weapon_info[best_weapon]:
                        best_weapon = weapon

                group_chars.append(DnD.Creature(f'group_{g}_{i}', abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                              'int': char['Int'], 'wis': char['Wis']}, 
                         ac=char.AC, hp=char.HP, attack_parameters=best_weapon, level=char['level'], alignment=f'group{g}' ))

            chars.append(group_chars)

        char_list[group].append(chars)


19 1 3
choice HD... 4.0
18 3 2
choice HD... 4.0
12 2 1
choice HD... 5.0
24 2 3
choice HD... 4.5
13 3 1
choice HD... 5.0
15 3 1
choice HD... 6.0
11 3 2
choice HD... 1.6666666666666667
8 0 1
choice HD... 4.0
44 4 3
choice HD... 8.0
12 1 2
choice HD... 3.3333333333333335
24 0 3
choice HD... 6.0
39 4 3
choice HD... 6.75
12 2 1
choice HD... 5.0
26 1 3
choice HD... 5.75
6 0 1
choice HD... 3.0
11 2 1
choice HD... 4.5
5 0 1
choice HD... 2.5
7 1 1
choice HD... 3.0
9 -1 1
choice HD... 5.0
8 2 1
choice HD... 3.0
14 2 2
choice HD... 3.3333333333333335
24 2 3
choice HD... 4.5
11 3 1
choice HD... 4.0
13 -1 3
choice HD... 4.0
9 1 1
choice HD... 4.0
26 4 3
choice HD... 3.5
8 2 1
choice HD... 3.0
9 1 1
choice HD... 4.0
10 2 1
choice HD... 4.0
10 2 1
choice HD... 4.0
7 -1 1
choice HD... 4.0
10 2 1
choice HD... 4.0
37 2 3
choice HD... 7.75
21 1 3
choice HD... 4.5
8 0 1
choice HD... 4.0
11 3 1
choice HD... 4.0
8 2 1
choice HD... 3.0
11 3 1
choice HD... 4.0
9 1 1
choice HD... 4.0
11 3 1
choice HD... 4.0
12

./DnD-Battler/DnD.py:383: UserWarning: Unfinished case to guess HD. so Defaulting hit dice to d8 instead
  warnings.warn('Unfinished case to guess HD. so Defaulting hit dice to d8 instead')  # TODO finish


9 1 1
choice HD... 4.0
12 2 1
choice HD... 5.0
28 2 3
choice HD... 5.5
15 1 2
choice HD... 4.333333333333333
13 3 1
choice HD... 5.0
9 -1 1
choice HD... 5.0
11 3 1
choice HD... 4.0
24 1 3
choice HD... 5.25
12 2 2
choice HD... 2.6666666666666665
7 -1 1
choice HD... 4.0
10 2 1
choice HD... 4.0
23 2 3
choice HD... 4.25
10 2 1
choice HD... 4.0
45 2 1
choice HD... 21.5
10 2 1
choice HD... 4.0
23 3 2
choice HD... 5.666666666666667
30 2 3
choice HD... 6.0
9 1 1
choice HD... 4.0
10 2 1
choice HD... 4.0
8 0 1
choice HD... 4.0
17 2 2
choice HD... 4.333333333333333
15 2 3
choice HD... 2.25
8 0 1
choice HD... 4.0
10 2 1
choice HD... 4.0
11 1 1
choice HD... 5.0
27 2 3
choice HD... 5.25
16 2 2
choice HD... 4.0
11 3 1
choice HD... 4.0
28 2 3
choice HD... 5.5
10 2 1
choice HD... 4.0
21 2 2
choice HD... 5.666666666666667
22 3 2
choice HD... 5.333333333333333
6 -1 1
choice HD... 3.5
6 0 1
choice HD... 3.0
28 2 3
choice HD... 5.5
9 1 1
choice HD... 4.0
30 3 3
choice HD... 5.25
10 2 1
choice HD... 4.0
11 

24 2 3
choice HD... 4.5
12 2 1
choice HD... 5.0
16 0 2
choice HD... 5.333333333333333
18 1 3
choice HD... 3.75
30 3 3
choice HD... 5.25
9 3 1
choice HD... 3.0
11 1 1
choice HD... 5.0
35 3 3
choice HD... 6.5
8 0 1
choice HD... 4.0
10 2 1
choice HD... 4.0
14 1 2
choice HD... 4.0
10 2 1
choice HD... 4.0
28 2 2
choice HD... 8.0
15 3 1
choice HD... 6.0
11 3 1
choice HD... 4.0
21 0 3
choice HD... 5.25
9 2 1
choice HD... 3.5
27 3 3
choice HD... 4.5
9 1 1
choice HD... 4.0
10 2 1
choice HD... 4.0
31 4 2
choice HD... 7.666666666666667
17 2 2
choice HD... 4.333333333333333
24 2 3
choice HD... 4.5
13 3 1
choice HD... 5.0
33 5 3
choice HD... 4.5
15 1 2
choice HD... 4.333333333333333
11 1 1
choice HD... 5.0
22 -2 3
choice HD... 7.0
30 2 3
choice HD... 6.0
22 3 2
choice HD... 5.333333333333333
24 1 3
choice HD... 5.25
28 2 3
choice HD... 5.5
10 2 1
choice HD... 4.0
8 0 1
choice HD... 4.0
8 0 1
choice HD... 4.0
18 1 2
choice HD... 5.333333333333333
10 2 1
choice HD... 4.0
12 4 1
choice HD... 4.0
9 1 1

6 1 2
choice HD... 1.3333333333333333
12 2 1
choice HD... 5.0
21 2 2
choice HD... 5.666666666666667
12 4 1
choice HD... 4.0
19 2 2
choice HD... 5.0
24 2 3
choice HD... 4.5
10 2 1
choice HD... 4.0
25 3 3
choice HD... 4.0
12 1 3
choice HD... 2.25
25 3 2
choice HD... 6.333333333333333
15 1 2
choice HD... 4.333333333333333
8 0 1
choice HD... 4.0
8 0 1
choice HD... 4.0
24 2 3
choice HD... 4.5
31 3 3
choice HD... 5.5
28 3 3
choice HD... 4.75
12 2 1
choice HD... 5.0
24 1 3
choice HD... 5.25
22 3 2
choice HD... 5.333333333333333
13 0 2
choice HD... 4.333333333333333
29 2 3
choice HD... 5.75
24 2 3
choice HD... 4.5
31 3 3
choice HD... 5.5
20 2 3
choice HD... 3.5
36 2 3
choice HD... 7.5
24 4 2
choice HD... 5.333333333333333
18 0 3
choice HD... 4.5
10 2 1
choice HD... 4.0
9 1 1
choice HD... 4.0
10 2 1
choice HD... 4.0
18 0 3
choice HD... 4.5
28 2 3
choice HD... 5.5
11 1 1
choice HD... 5.0
29 1 3
choice HD... 6.5
11 2 1
choice HD... 4.5
20 1 2
choice HD... 6.0
10 2 1
choice HD... 4.0
14 4 1
choice

...

148 3 16
choice HD... 5.882352941176471
150 3 18
choice HD... 5.052631578947368
176 1 16
choice HD... 9.411764705882353
145 3 16
choice HD... 5.705882352941177
182 4 18
choice HD... 5.7894736842105265
195 3 18
choice HD... 7.421052631578948
102 2 16
choice HD... 4.117647058823529
150 3 18
choice HD... 5.052631578947368
146 4 18
choice HD... 3.8947368421052633
216 5 17
choice HD... 7.277777777777778
164 4 16
choice HD... 5.882352941176471
167 3 16
choice HD... 7.0
217 4 18
choice HD... 7.631578947368421
105 -1 18
choice HD... 6.473684210526316
164 4 16
choice HD... 5.882352941176471
198 3 17
choice HD... 8.166666666666666
198 3 17
choice HD... 8.166666666666666
89 1 17
choice HD... 4.0
176 2 17
choice HD... 7.888888888888889
102 2 16
choice HD... 4.117647058823529
174 4 17
choice HD... 5.888888888888889
117 3 16
choice HD... 4.0588235294117645
176 3 16
choice HD... 7.529411764705882
194 3 16
choice HD... 8.588235294117647
208 3 16
choice HD... 9.411764705882353
155 3 16
choice HD... 6.2

129 2 18
choice HD... 4.894736842105263
129 2 18
choice HD... 4.894736842105263
105 0 16
choice HD... 6.176470588235294
117 3 16
choice HD... 4.0588235294117645
130 2 16
choice HD... 5.764705882352941
160 2 16
choice HD... 7.529411764705882
198 3 17
choice HD... 8.166666666666666
229 4 16
choice HD... 9.705882352941176
116 1 16
choice HD... 5.882352941176471
160 3 16
choice HD... 6.588235294117647
168 4 16
choice HD... 6.117647058823529
170 3 18
choice HD... 6.105263157894737
110 1 16
choice HD... 5.529411764705882
191 6 16
choice HD... 5.588235294117647
195 3 18
choice HD... 7.421052631578948
216 4 18
choice HD... 7.578947368421052
198 3 17
choice HD... 8.166666666666666
216 5 17
choice HD... 7.277777777777778
114 2 18
choice HD... 4.105263157894737
20 0 18
choice HD... 1.0526315789473684
149 4 17
choice HD... 4.5
118 2 16
choice HD... 5.0588235294117645
120 1 20
choice HD... 4.761904761904762
124 0 20
choice HD... 5.904761904761905
280 2 20
choice HD... 11.428571428571429
10 1 20
cho

182 4 20
choice HD... 4.857142857142857
163 3 20
choice HD... 4.904761904761905
285 4 20
choice HD... 9.761904761904763
245 5 20
choice HD... 6.904761904761905
163 3 20
choice HD... 4.904761904761905
143 2 20
choice HD... 4.904761904761905
143 2 20
choice HD... 4.904761904761905
200 2 20
choice HD... 7.619047619047619
133 1 20
choice HD... 5.380952380952381
195 4 20
choice HD... 5.476190476190476
123 1 20
choice HD... 4.904761904761905
240 2 20
choice HD... 9.523809523809524
163 3 20
choice HD... 4.904761904761905
359 5 20
choice HD... 12.333333333333334
163 3 20
choice HD... 4.904761904761905
160 0 20
choice HD... 7.619047619047619
189 4 20
choice HD... 5.190476190476191
125 1 20
choice HD... 5.0
212 3 20
choice HD... 7.238095238095238
103 0 20
choice HD... 4.904761904761905
123 1 20
choice HD... 4.904761904761905
300 3 20
choice HD... 11.428571428571429
103 0 20
choice HD... 4.904761904761905
133 1 20
choice HD... 5.380952380952381
300 3 20
choice HD... 11.428571428571429
150 1 20
ch

143 2 20
choice HD... 4.904761904761905
175 1 19
choice HD... 7.8
260 5 20
choice HD... 7.619047619047619
200 2 20
choice HD... 7.619047619047619
156 2 20
choice HD... 5.523809523809524
245 5 20
choice HD... 6.904761904761905
155 2 20
choice HD... 5.476190476190476
248 2 20
choice HD... 9.904761904761905
218 4 20
choice HD... 6.571428571428571
301 7 20
choice HD... 7.666666666666667
182 5 20
choice HD... 3.9047619047619047
274 7 20
choice HD... 6.380952380952381
243 7 20
choice HD... 4.904761904761905
240 4 20
choice HD... 7.619047619047619
120 1 20
choice HD... 4.761904761904762
163 2 20
choice HD... 5.857142857142857
143 2 20
choice HD... 4.904761904761905
285 4 20
choice HD... 9.761904761904763
144 2 20
choice HD... 4.9523809523809526
156 2 20
choice HD... 5.523809523809524
271 5 20
choice HD... 8.142857142857142
195 4 20
choice HD... 5.476190476190476
103 0 20
choice HD... 4.904761904761905
225 3 20
choice HD... 7.857142857142857
183 3 20
choice HD... 5.857142857142857
200 2 20
cho

224 3 20
choice HD... 7.809523809523809
103 0 20
choice HD... 4.904761904761905
183 2 20
choice HD... 6.809523809523809
359 6 20
choice HD... 11.380952380952381
143 2 20
choice HD... 4.904761904761905
124 0 20
choice HD... 5.904761904761905
92 2 20
choice HD... 2.4761904761904763
121 1 20
choice HD... 4.809523809523809
301 7 20
choice HD... 7.666666666666667
224 5 20
choice HD... 5.904761904761905
144 2 20
choice HD... 4.9523809523809526
209 3 20
choice HD... 7.095238095238095
240 4 20
choice HD... 7.619047619047619
167 4 20
choice HD... 4.142857142857143
103 0 20
choice HD... 4.904761904761905
138 3 20
choice HD... 3.7142857142857144
124 0 20
choice HD... 5.904761904761905
225 3 20
choice HD... 7.857142857142857
166 3 20
choice HD... 5.0476190476190474
202 4 20
choice HD... 5.809523809523809
295 5 20
choice HD... 9.285714285714286
163 3 20
choice HD... 4.904761904761905
143 2 20
choice HD... 4.904761904761905
203 5 20
choice HD... 4.904761904761905
284 5 20
choice HD... 8.761904761904

In [38]:
def calc_avg_roll(die):
    split_die = die.split('d')
    die_nums = split_die[1].split('+')
    if split_die[0] == '':
        num_die = 1
    else: 
        num_die = int(split_die[0])
        
    max_roll = int(die_nums[0])
    if len(die_nums) == 1:
        modifier = 0
    else: 
        modifier = int(die_nums[1])
    
    
    
    return  (((max_roll + 1 ) / 2 ) + modifier) * num_die

# Scales a matrix either using given parameters, or if none scales it to be standard normal.
# TODO: what to do if std is 0??
def scale_matrix(X, mean=None, var=None):
    if mean is None:
        mean = np.mean(X, axis=0)
    
    if var is None:
        var = np.var(X, axis=0)
        
    std = np.sqrt(var)
        
    for i in range(len(X)):
        for j in range(len(X[0])):
            if std[j] == 0:
                std[j] = 1
            X[i][j] = (X[i][j] - mean[j]) / std[j]
    
    return X, mean, var

def k_fold_validation(X, y, C, gamma, k):
    accuracy = defaultdict()
    for c in C:
        for g in gamma:
            tmp = list(zip(X, y))
            random.shuffle(tmp)
            X_shfl, y_shfl = zip(*tmp) 
            X_spl = np.array_split(X_shfl, k)
            y_spl = np.array_split(y_shfl, k)
            
            for i in range(len(X_spl)):
                X_train = [item for li in X_spl[:i] for item in li] + [item for li in X_spl[i + 1:] for item in li]
                X_validate = X_spl[i]
                
                y_train = [item for li in y_spl[:i] for item in li] + [item for li in y_spl[i + 1:] for item in li]
                y_validate = y_spl[i]
                
                svm = SVR(C=c, gamma=g)
                svm.fit(X_train, y_train)
                accuracy[(c, g)] = svm.score(X_validate, y_validate)
            
    return accuracy

In [16]:
number_simulations = 1000
df = pd.DataFrame(columns=['level', 'proficiency', 'able', 'strength', 
                           'con', 'wis', 'intelligence', 'cha', 'hd', 'hp', 'ac', 'initiative', 'healing_spells',
                           'buff_spells', 'attack_bonus', 'weapon_damage', 'winrate'])

for level_group in level_groups:
    for chars in char_list[level_group]:
        # Currently hardcoded for 1 char / group, 2 groups
        arena = DnD.Encounter(*chars[0], *chars[1])
        res = json.loads(arena.go_to_war(number_simulations).json())
        
        
        battle_results = {
            res['team_names'][0]: {
                'victories': res['team_victories'][0],
                'close': res['team_close'][0],
            },

            res['team_names'][1]: {
                'victories': res['team_victories'][1],
                'close': res['team_close'][1]
            }
        }

        battle_results[chars[0][0].__dict__['alignment']]['chars'] = [c.__dict__ for c in chars[0]]
        battle_results[chars[1][0].__dict__['alignment']]['chars'] = [c.__dict__ for c in chars[1]]
        
        group0_char_dict = battle_results['group0']['chars'][0]
        group0_char_info = {
            'level': group0_char_dict['level'],
            'proficiency': group0_char_dict['proficiency'],
            'able': group0_char_dict['able'],
            'strength': group0_char_dict['abilities']['str'],
            'con': group0_char_dict['abilities']['con'],
            'wis': group0_char_dict['abilities']['wis'],
            'intelligence': group0_char_dict['abilities']['int'],
            'cha': group0_char_dict['abilities']['cha'],
            'hd': calc_avg_roll(str(group0_char_dict['hd'])),
            'hp': group0_char_dict['hp'],
            'ac': group0_char_dict['ac'],
            'initiative': calc_avg_roll(str(group0_char_dict['initiative'])),
            'healing_spells': group0_char_dict['healing_spells'],
            'buff_spells': group0_char_dict['buff_spells'],
            'attack_bonus': group0_char_dict['attack_parameters'][0][2],
            'weapon_damage': group0_char_dict['attack_parameters'][0][3]
        }
        
        group1_char_dict = battle_results['group1']['chars'][0]
        group1_char_info = {
            'level': group1_char_dict['level'],
            'proficiency': group1_char_dict['proficiency'],
            'able': group1_char_dict['able'],
            'strength': group1_char_dict['abilities']['str'],
            'con': group1_char_dict['abilities']['con'],
            'wis': group1_char_dict['abilities']['wis'],
            'intelligence': group1_char_dict['abilities']['int'],
            'cha': group1_char_dict['abilities']['cha'],
            'hd': calc_avg_roll(str(group1_char_dict['hd'])),
            'hp': group1_char_dict['hp'],
            'ac': group1_char_dict['ac'],
            'initiative': calc_avg_roll(str(group1_char_dict['initiative'])),
            'healing_spells': group1_char_dict['healing_spells'],
            'buff_spells': group1_char_dict['buff_spells'],
            'attack_bonus': group1_char_dict['attack_parameters'][0][2],
            'weapon_damage': group1_char_dict['attack_parameters'][0][3]
        }

        diff = {}

        for key in group0_char_info.keys():
            if type(group0_char_info[key]) is list:
                diff[key] = [group0_char_info[key][i] - group1_char_info[key][i] for i in range(len(group0_char_info[key]))]
            else:
                diff[key] = group0_char_info[key] - group1_char_info[key]
        
    
        diff['winrate'] = battle_results['group0']['victories'] / number_simulations
        
        df = df.append(diff, ignore_index=True)

df.to_csv('./simulations.csv', index = False)
print(df)
    
# Attack params are attack bonus, weapon damage

# for key in test['group1']['chars'][0].__dict__.keys():
#     print(key + ": " + str(test['group1']['chars'][0].__dict__[key]) + "\n")

# arena = DnD.Encounter(*char_list['1-3'][1][0], *char_list['1-3'][1][1])
# res = json.loads(arena.go_to_war(number_simulations).json())   

# test = {
#     res['team_names'][0]: {
#         'victories': res['team_victories'][0],
#         'close': res['team_close'][0],
#     },
    
#     res['team_names'][1]: {
#         'victories': res['team_victories'][1],
#         'close': res['team_close'][1]
#     }
# }

# test[char_list['1-3'][1][0][0].__dict__['alignment']]['chars'] = char_list['1-3'][1][0]
# test[char_list['1-3'][1][1][0].__dict__['alignment']]['chars'] = char_list['1-3'][1][1]


# char1_dict = test['group0']['chars'][0].__dict__
# char1_info = {
#     'level': char1_dict['level'],
#     'proficiency': char1_dict['proficiency'],
#     'able': char1_dict['able'],
#     'strength': char1_dict['abilities']['str'],
#     'con': char1_dict['abilities']['con'],
#     'wis': char1_dict['abilities']['wis'],
#     'intelligence': char1_dict['abilities']['int'],
#     'cha': char1_dict['abilities']['cha'],
#     'hd': calc_avg_roll(str(char1_dict['hd'])),
#     'hp': char1_dict['hp'],
#     'ac': char1_dict['ac'],
#     'initiative': calc_avg_roll(str(char1_dict['initiative'])),
#     'healing_spells': char1_dict['healing_spells'],
#     'attack_bonus': char1_dict['attack_parameters'][0][2],
#     'weapon_damage': char1_dict['attack_parameters'][0][3]
# }

# char2_dict = test['group1']['chars'][0].__dict__
# char2_info = {
#     'level': char2_dict['level'],
#     'proficiency': char2_dict['proficiency'],
#     'able': char2_dict['able'],
#     'strength': char2_dict['abilities']['str'],
#     'con': char2_dict['abilities']['con'],
#     'wis': char2_dict['abilities']['wis'],
#     'intelligence': char2_dict['abilities']['int'],
#     'cha': char2_dict['abilities']['cha'],
#     'hd': calc_avg_roll(str(char2_dict['hd'])),
#     'hp': char2_dict['hp'],
#     'ac': char2_dict['ac'],
#     'initiative': calc_avg_roll(str(char2_dict['initiative'])),
#     'healing_spells': char2_dict['healing_spells'],
#     'buff_spells': char2_dict['buff_spells'],
#     'attack_bonus': char2_dict['attack_parameters'][0][2],
#     'weapon_damage': char2_dict['attack_parameters'][0][3]
# }

# diff = {}

# for key in char1_info.keys():
#     if type(char1_info[key]) is list:
#         diff[key] = [char1_info[key][i] - char2_info[key][i] for i in range(len(char1_info[key]))]
#     else:
#         diff[key] = char1_info[key] - char2_info[key]
        
# print(diff)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(clean_df.iloc[0])
    
# weapons = clean_df.iloc[0]['weapons']
# for weapon in weapons:
#     print(weapons[weapon]['processedWeapon'])
    
# clean_df
# pd.json_normalize(clean_df.iloc[0]['weapons'])

      level  proficiency  able  strength  con  wis  intelligence  cha   hd  \
0       1.0          1.0   0.0      -5.0 -4.0 -1.0           7.0  6.0 -2.0   
1      -2.0         -1.0   0.0       8.0  0.0 -3.0           0.0  0.0  0.0   
2       0.0          0.0   0.0      -1.0 -1.0  0.0           0.0  3.0  0.0   
3       1.0          0.0   0.0       8.0  6.0  1.0          -4.0 -2.0  3.0   
4       1.0          1.0   0.0       4.0  7.0 -8.0           2.0 -2.0  3.0   
...     ...          ...   ...       ...  ...  ...           ...  ...  ...   
2995    1.0          0.0   0.0      -3.0  2.0 -4.0           0.0 -5.0  1.0   
2996    0.0          0.0   0.0       6.0  0.0  4.0           8.0  0.0  0.0   
2997    0.0          0.0   0.0      -1.0  8.0 -2.0          -6.0  7.0  4.0   
2998    0.0          0.0   0.0     -10.0 -6.0 -4.0           2.0  0.0 -3.0   
2999    0.0          0.0   0.0     -12.0  4.0 -6.0           9.0 -5.0  2.0   

         hp    ac  initiative  healing_spells  buff_spells  att

In [40]:
y = df.winrate
X = df.drop('winrate', axis=1)

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)
X_trz, mean_tr, var_tr = scale_matrix(X_tr.values.tolist())
X_tez, _, _ = scale_matrix(X_te.values.tolist(), mean_tr, var_tr)

C = [0.1, 1, 10]
gamma = [0.001, 0.1, 1]
k = 5
accuracy = k_fold_validation(X_trz, y_tr.values.tolist(), C, gamma, k)
C, gamma = max(accuracy, key=accuracy.get)

svm = SVR(C=C, gamma=gamma)
svm.fit(X_trz, y_tr)
test_acc = svm.score(X_tez, y_te)
print(f'(C: {C}, gamma: {gamma}) yields the highest training accuracy of {accuracy[(C, gamma)] * 100}%')
print(f'Final testing accuracy: {test_acc * 100}%')

(C: 1, gamma: 0.1) yields the highest training accuracy of 89.88757108250051%
Final testing accuracy: 91.02157944924217%


[-4.82109085e-02  3.75467541e-01  6.83410793e-01  2.04879601e-01
  4.81832116e-01  8.55641655e-01  2.77511104e-01  3.59331350e-01
  7.67273462e-01 -2.72278743e-02 -7.61106691e-03 -2.00043354e-01
  9.51506949e-01  8.84725350e-01  5.15388015e-02  5.09942038e-01
  9.56011652e-01  9.89701853e-01  9.86091855e-01  2.91184126e-01
  6.24098214e-01  8.19105564e-01  8.53504726e-01  3.36276014e-01
  9.24806119e-01  7.60277651e-01  9.60463754e-01  5.98973792e-01
  1.48655633e-01 -8.53835174e-02  9.06313705e-01  3.56316144e-01
  9.85386010e-01  1.04549329e+00  8.65588686e-01  9.47836188e-01
  6.81297887e-01  6.78452121e-01 -1.07855086e-02  6.31129808e-01
  1.06563075e+00  1.05932616e+00  9.35075448e-01  1.08368053e+00
 -2.27170541e-02  8.97633951e-03  4.70265879e-01  8.58801231e-01
  7.80874194e-01  8.86134261e-01  7.19433688e-02  7.06910598e-01
  8.72311549e-01  9.63603356e-01  3.88580858e-02  1.91445658e-01
  1.46730851e-01  4.11713564e-01  9.71218539e-01 -1.62156459e-01
  1.09718377e+00  8.91274